<a href="https://colab.research.google.com/github/elilaird/Stats202-FinalProject/blob/master/Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


#Read in data


In [0]:
study_A = pd.read_csv("/content/drive/My Drive/Stats202_FinalProject/Data/Study_A.csv")
study_B = pd.read_csv("/content/drive/My Drive/Stats202_FinalProject/Data/Study_B.csv")
study_C = pd.read_csv("/content/drive/My Drive/Stats202_FinalProject/Data/Study_C.csv")
study_D = pd.read_csv("/content/drive/My Drive/Stats202_FinalProject/Data/Study_D.csv")
study_E = pd.read_csv("/content/drive/My Drive/Stats202_FinalProject/Data/Study_E.csv")

targets = pd.read_csv("/content/drive/My Drive/Stats202_FinalProject/Data/sample_submission_PANSS.csv")

#merge datasets
full_set = pd.concat([study_A, study_B, study_C, study_D])
full_set.drop(columns = "LeadStatus", inplace = True)
full_set = pd.concat([full_set, study_E])
#full_set = study_E[study_E["PatientID"].isin(targets["PatientID"])]

full_set = full_set.reset_index(drop=True)

# Replace TxGroup with numerical values and drop irrelevant columns

In [0]:
mapping = {"Treatment" : 1, "Control": 0}
full_set["TxGroup"].replace(mapping, inplace=True)
important_columns = full_set.drop(columns = ["Country", "Study", "SiteID", "RaterID", "P1", "P2", "P3","P4", "P5", "P6", "P7","N1", "N2", "N3","N4", "N5", "N6", "N7","G1", "G2", "G3","G4", "G5", "G6", "G7","G8", "G9", "G10","G11", "G12", "G13", "G14", "G15", "G16", "AssessmentiD"])

# ONE HOT ENCODING

In [0]:
categorical_data = ["PatientID"]
def one_hot(df, cols):
    for each in cols:
        dummies = pd.get_dummies(df[each], prefix=each, drop_first=False)
       
        df = pd.concat([df, dummies], axis=1)
    return df
important_columns = one_hot(important_columns, categorical_data)
#output_set = one_hot(output_set, categorical_data)

# TRAIN TEST SPLIT

In [0]:
from sklearn.model_selection import train_test_split
length = important_columns.shape[1]
X = important_columns.drop(columns = ["PANSS_Total", "PatientID"], axis=1)
y = important_columns["PANSS_Total"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)
X_train.head()

,TxGroup,VisitDay,PatientID_10001,PatientID_10002,PatientID_10003,PatientID_10004,PatientID_10005,PatientID_10006,PatientID_10007,PatientID_10008,PatientID_10009,PatientID_10010,PatientID_10011,PatientID_10012,PatientID_10013,PatientID_10014,PatientID_10015,PatientID_10016,PatientID_10017,PatientID_10018,PatientID_10019,PatientID_10020,PatientID_10021,PatientID_10022,PatientID_10023,PatientID_10024,PatientID_10025,PatientID_10026,PatientID_10027,PatientID_10028,PatientID_10029,PatientID_10030,PatientID_10031,PatientID_10032,PatientID_10033,PatientID_10034,PatientID_10035,PatientID_10036,PatientID_10037,PatientID_10038,...,PatientID_50474,PatientID_50475,PatientID_50476,PatientID_50477,PatientID_50478,PatientID_50479,PatientID_50480,PatientID_50481,PatientID_50482,PatientID_50483,PatientID_50484,PatientID_50485,PatientID_50486,PatientID_50487,PatientID_50488,PatientID_50489,PatientID_50490,PatientID_50491,PatientID_50492,PatientID_50493,PatientID_50494,PatientID_50495,PatientID_50496,PatientID_50497,PatientID_50498,PatientID_50499,PatientID_50500,PatientID_50501,PatientID_50502,PatientID_50503,PatientID_50504,PatientID_50505,PatientID_50506,PatientID_50507,PatientID_50508,PatientID_50509,PatientID_50510,PatientID_50511,PatientID_50512,PatientID_50513
15895,0,162,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
18841,1,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1866,0,54,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16668,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10317,0,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# XGBOOST

In [0]:
import xgboost as xgb
reg = xgb.XGBRegressor(n_estimators=8000, seed = 42) 
reg.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)], eval_metric = "rmse",
        early_stopping_rounds=100,
       verbose=False)

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[23:47:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


## Result XGBoost

In [0]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
predictions = reg.predict(X_test)
print("Training Error: " + str(mean_squared_error(y_train, reg.predict(X_train))))
print("TestError " + str(mean_squared_error(y_test,predictions)))


# Select Data for Kaggle Competition

In [0]:
patients = full_set[full_set["PatientID"].isin(targets["PatientID"])]
patients.shape
output_set = patients[["PatientID", "TxGroup"]]
output_set.drop_duplicates(inplace = True)
output_set.head()
ids = output_set["PatientID"].tolist()


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [0]:
kaggle_data = important_columns[important_columns["PatientID"].isin(ids)]
kaggle_data["VisitDay"] = 120
prediction_set = kaggle_data.drop(columns = ["PANSS_Total"]  ,axis = 1)
prediction_set.drop_duplicates(inplace = True)
final_prediction = prediction_set.drop(columns = ["PatientID"]  ,axis = 1)
prediction_set



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

,PatientID,TxGroup,VisitDay,InteractionMultiplication,InteractionDivison,PatientID_10001,PatientID_10002,PatientID_10003,PatientID_10004,PatientID_10005,PatientID_10006,PatientID_10007,PatientID_10008,PatientID_10009,PatientID_10010,PatientID_10011,PatientID_10012,PatientID_10013,PatientID_10014,PatientID_10015,PatientID_10016,PatientID_10017,PatientID_10018,PatientID_10019,PatientID_10020,PatientID_10021,PatientID_10022,PatientID_10023,PatientID_10024,PatientID_10025,PatientID_10026,PatientID_10027,PatientID_10028,PatientID_10029,PatientID_10030,PatientID_10031,PatientID_10032,PatientID_10033,PatientID_10034,PatientID_10035,...,PatientID_50474,PatientID_50475,PatientID_50476,PatientID_50477,PatientID_50478,PatientID_50479,PatientID_50480,PatientID_50481,PatientID_50482,PatientID_50483,PatientID_50484,PatientID_50485,PatientID_50486,PatientID_50487,PatientID_50488,PatientID_50489,PatientID_50490,PatientID_50491,PatientID_50492,PatientID_50493,PatientID_50494,PatientID_50495,PatientID_50496,PatientID_50497,PatientID_50498,PatientID_50499,PatientID_50500,PatientID_50501,PatientID_50502,PatientID_50503,PatientID_50504,PatientID_50505,PatientID_50506,PatientID_50507,PatientID_50508,PatientID_50509,PatientID_50510,PatientID_50511,PatientID_50512,PatientID_50513
20948,50002,1,120,7319.294299,2.102103,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20952,50003,1,120,7319.294299,2.102103,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20957,50004,0,120,7319.294299,2.102103,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20966,50006,1,120,7319.294299,2.102103,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20971,50007,1,120,7319.294299,2.102103,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20976,50008,1,120,7319.294299,2.102103,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20981,50009,1,120,7319.294299,2.102103,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20990,50012,0,120,7319.294299,2.102103,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20995,50013,0,120,7319.294299,2.102103,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
21001,50015,0,120,7319.294299,2.102103,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
result = reg.predict(final_prediction)

# CSV OUTPUT RESULTS

In [0]:
prediction_set["PANSS_Total"] = result
csv_output = prediction_set[["PatientID", "PANSS_Total"]]
csv_output.head()


,PatientID,PANSS_Total
20948,50002,59.75106
20952,50003,59.75106
20957,50004,59.75106
20966,50006,59.75106
20971,50007,59.75106


In [0]:
csv_output.to_csv(r"/content/drive/My Drive/Stats202_FinalProject/forecasting_results.csv", header = True, index = False)